In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

import re
import string
import operator

# 读取数据集

In [ ]:
def loadDataSet(filePath):
    articles, labels = [], []
    with open(filePath, 'r', encoding="utf-8") as f:
        for line in f.readlines():
            part = line.split("\t")
            #这里已经将标签转换为大写，数据转换成小写了
            label, article = part[0].upper().strip(), part[2].lower().replace("<sssss>", " ").strip().split(" ")
            articles.append(article)
            labels.append(label)
    return articles, labels
        

train_articles, train_labels = loadDataSet(".\\data\\MulLabelTrain.ss")
test_articles, test_labels = loadDataSet(".\\data\\MulLabelTest.ss")

len(train_articles), len(test_articles)

In [ ]:
print(train_articles[0], "\n\n", train_labels[0])

# 去除高频词和低频词得到所有词列表

In [ ]:
all_word_train_mix = [j for i in train_articles for j in i]
all_word_test_mix = [j for i in test_articles for j in i]

len(all_word_train_mix), len(all_word_test_mix)

In [ ]:
all_word_mix = all_word_train_mix + all_word_test_mix
len(all_word_mix)

In [ ]:
def getCountDF(in_list):  
    #根据Counter统计的词频初始化df
    df = DF.from_dict(Counter(in_list), orient='index').reset_index()
    #根据count值降序排序
    df = df.rename(columns={'index':'words',
                                  0:'count'}).sort_values(["count"],ascending=False).reset_index(drop=True)
    return df

all_word_mix_df = getCountDF(all_word_mix)
all_word_mix_df

In [ ]:
low, high = 500, 50000
df2 = all_word_mix_df[np.logical_and(low <= all_word_mix_df['count'], all_word_mix_df['count'] <= high)].reset_index(drop=True) 
df2.shape
df2

In [ ]:
df3 = df2.drop(["count"], axis=1)
df3.head(20)

## 去除数字以及其他无用词汇

In [ ]:
df3["words"].shape
df4 = df3["words"].apply(lambda x: re.sub("[0-9||\.||\?||!]*", "", x))
df4 = df4[df4 != ""]
df4.shape

In [ ]:
all_word_unique = df4.values
all_word_unique

## 测试-1

检查训练集和测试集的词的不重复数

In [ ]:
# all_word_train = set(all_word_train_mix)
# all_word_test = set(all_word_test_mix)

# len(all_word_train), len(all_word_test), (len(all_word_train) - len(all_word_test))

# diff1 = all_word_train.difference(all_word_test)
# diff2 = all_word_test.difference(all_word_train)

# print("number of words in train but not in test: ", len(diff1))
# print("number of words in test but not in train: ", len(diff2))

# all_word = all_word_train.union(all_word_test)
# len(all_word)

## 测试-2
检查词频分布

In [ ]:
# def show_common_word_rate(lst, k):
#     word_counter = Counter(lst)
#     cnt = 0
#     tot_cnt = len(lst)
#     common_pair = word_counter.most_common(k)
#     for key, val in common_pair:
#         cnt += val
#     print("max frequent word and count: ", common_pair[0])
#     print("min frequent word and count: ", common_pair[-1])
#     print("frequent word count: ", cnt)
#     print("total word count: ", tot_cnt)
#     print("remain word count: ", tot_cnt-cnt)
#     print("rate:%.5f%%" % (cnt/tot_cnt*100))

# #测试
# #show_common_word_rate([1,2,3,4,2,2], 2)
# show_common_word_rate(all_word_mix, 500)
# show_common_word_rate(all_word_mix, 50000)

# 得到TF-IDF矩阵

In [ ]:
def getTF(dataSet, allWords):
    '''得到输入数据集的TF矩阵'''
    def safeDivide(a, b):
        return a/b if b!=0 else 0
    
    TF=[]
    for index in tnrange(len(dataSet)):
        TF.append([])
        wordCounter = Counter(dataSet[index])
        for word in allWords:
            TF[index].append(safeDivide(wordCounter.get(word,0), len(dataSet[index])))
    return arr(TF)

train_TF = getTF(train_articles, all_word_unique)
test_TF = getTF(test_articles, all_word_unique)

train_TF.shape, test_TF.shape

## 保存TF矩阵(**修改路径位置**)

In [ ]:
dirPath = "E:\\Code\\_largeData\\Github--Open-Course-Learning--A04\\Project\\multiclass classification\\data preprocessed\\tf-idf"
dirPath += "\\without removing stopwords"

if not os.path.exists(dirPath):
    os.makedirs(dirPath)

    
train_tf_DF = DF(train_TF)
train_tf_DF.columns = all_word_unique

test_tf_DF = DF(test_TF)
test_tf_DF.columns = all_word_unique

train_tf_DF.to_csv(dirPath + '\\train_tf.csv', index=False, header=True)
test_tf_DF.to_csv(dirPath + '\\test_tf.csv', index=False, header=True)

DF(all_word_unique).to_csv(dirPath + '\\all_word_unique.csv', index=False, header=True)

这里需要注意，有一些样本去掉停用词后整个都没有数据了，这时候就默认TF那一列都为0了。

## 计算TF-IDF矩阵

In [ ]:
import math

def getIDF(dataSet, allWords):
    '''得到输入数据集的IDF矩阵'''
    def calcIDF(num):
        '''计算对应数据集的单词的IDF值'''
        return math.log(len(dataSet)/(1+num), 2)
    
    IDF=[]
    for i in tnrange(len(allWords)):
        cnt = 0
        #计算词在每个文档出现的次数
        for doc in dataSet: 
            if allWords[i] in doc:
                cnt += 1
        IDF.append(calcIDF(cnt))
    return arr(IDF)

train_IDF = getIDF(train_articles, all_word_unique)
test_IDF = getIDF(test_articles, all_word_unique)

train_IDF.shape, test_IDF.shape

In [ ]:
import math

def getIDF(dataSet, allWords):
    '''得到输入数据集的IDF矩阵'''
    def calcIDF(num):
        '''计算对应数据集的单词的IDF值'''
        return math.log(len(dataSet)/(1+num), 2)
    
    IDF=[]
    for i in tnrange(len(allWords)):
        cnt = 0
        #计算词在每个文档出现的次数
        for doc in dataSet: 
            if allWords[i] in doc:
                cnt += 1
        IDF.append(calcIDF(cnt))
    return arr(IDF)

train_IDF = getIDF(train_articles, all_word_unique)
test_IDF = getIDF(test_articles, all_word_unique)

train_IDF.shape, test_IDF.shape

In [ ]:
train_TFIDF = train_TF * train_IDF
test_TFIDF = test_TF * test_IDF

train_TFIDF.shape, test_TFIDF.shape

### 保存TF-IDF矩阵

In [ ]:
# train_tfidf = DF(train_TFIDF)
# train_tfidf.columns = all_word_unique

# test_tfidf = DF(test_TFIDF)
# test_tfidf.columns = all_word_unique

# train_tfidf.to_csv(dirPath + '\\train_tfidf.csv', index=False, header=True)
# test_tfidf.to_csv(dirPath + '\\test_tfidf.csv', index=False, header=True)

## 划分数据集

In [ ]:
#划分比例
splitRate = 0.3
#划分的数目
splitNum = int(train_TFIDF.shape[0]*splitRate) 
#得到 训练集 和验证集
trainSet = train_TFIDF[:-splitNum]
validateSet = train_TFIDF[-splitNum:]

trainSet.shape, validateSet.shape

In [ ]:
train_labels = arr(train_labels)
test_labels = arr(test_labels)

trainSetLabel = train_labels[:-splitNum]
validateSetLabel = train_labels[-splitNum:]

trainSetLabel.shape, validateSetLabel.shape

In [ ]:
DF(trainSetLabel[0:20]).replace("LOW",0).replace("MID",1).replace("HIG",2)

## 保存结果

In [ ]:
DF(trainSet).to_csv(dirPath + '\\train.csv', index=False, header=False)
DF(validateSet).to_csv(dirPath + '\\validate.csv', index=False, header=False)
DF(test_TFIDF).to_csv(dirPath + '\\test.csv', index=False, header=False)

输出标签映射为数字。

In [ ]:
trainSetLabel = DF(trainSetLabel).replace("LOW",0).replace("MID",1).replace("HIG",2)
validateSetLabel = DF(validateSetLabel).replace("LOW",0).replace("MID",1).replace("HIG",2)

DF(trainSetLabel).to_csv(dirPath + '\\train_label.csv', index=False, header=False)
DF(validateSetLabel).to_csv(dirPath + '\\validate_label.csv', index=False, header=False)